In [1]:
def episode_to_df(df, episode=0):
    far_x=list()
    end_y=list()
    end_z=list()
    sample_df = pd.DataFrame()
    actions = df.loc['actions']
    drone_observations_df = df.loc['drone_ob']
    latest_attack_index= 0
    try:
        attack_start = actions.index(1)
    except:
        attack_start = len(actions)
    if attack_start > latest_attack_index:
        latest_attack_index = attack_start
#         latest_attack_episode = episode
    sample_df['action'] = actions
    sample_df['reward'] = df.loc['rewards']
    sample_df['reward_acum'] = sample_df['reward'].cumsum()
    episode_length = sample_df.shape[0]
    
    sample_df['step'] = list(range(episode_length))
    x=list()
    y=list()
    z=list()
    roll = list()
    pitch = list()
    yaw = list()
    # linear velocity
    vx = list()
    vy = list()
    vz = list()
    # angular velocity
    wx = list()
    wy = list()
    wz = list()
    # motors
    m1 = list()
    m2 = list()
    m3 = list()
    m4 = list()

    for i in range(len(drone_observations_df)):
    # print(drone_observations_df.iloc[1][0])
        drone_state=drone_observations_df[i]
        position=drone_state[0:3]
        quaternion=drone_state[3:7]
        rpy=drone_state[7:10]
        linear_v=drone_state[10:13]
        angular_v=drone_state[13:16]
        motors=drone_state[16:20]
        
        x.append(position[0])
        y.append(position[1])
        z.append(position[2])
        roll.append(rpy[0])
        pitch.append(rpy[1])
        yaw.append(rpy[2])
        m1.append(motors[0])
        m2.append(motors[1])
        m3.append(motors[2])
        m4.append(motors[3])
        # additional observations
        vx.append(linear_v[0])
        vy.append(linear_v[1])
        vz.append(linear_v[2])
        wx.append(angular_v[0])
        wy.append(angular_v[1])
        wz.append(angular_v[2])
        
        
    sample_df['x'] = x
    sample_df['y'] = y
    sample_df['z'] = z
    sample_df['roll'] = roll
    sample_df['pitch'] = pitch
    sample_df['yaw'] = yaw
    sample_df['episode'] = [episode]*episode_length
    sample_df['m1'] = m1
    sample_df['m2'] = m2
    sample_df['m3'] = m3
    sample_df['m4'] = m4
    # adding velocity to the dataframe
    sample_df['vx'] = vx
    sample_df['vy'] = vy
    sample_df['vz'] = vz
    sample_df['wx'] = wx
    sample_df['wy'] = wy
    sample_df['wz'] = wz
    
    # filling up small_df 
    min_x = min(x)
    max_x = max(x)
    farX = max_x if abs(max_x) > abs(min_x) else min_x
    far_x.append(farX)
    end_y.append(y[-1])
    end_z.append(z[-1])
    
    return sample_df

In [2]:
import pandas as pd
import plotly.express as px
from plotly.offline import iplot, init_notebook_mode


file_name = 'results_penalty_failsafe.json'
df_explore = pd.read_json(file_name)
print(df_explore.shape)
print(df_explore)

(4, 13)
                                                    1   \
goal      [-49.36999272476488, -414.32762319983567, 0]   
actions                                             []   
rewards                                             []   
drone_ob                                            []   

                                                         2   \
goal            [47.50568661314894, -447.22409773585053, 0]   
actions   [0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...   
rewards   [-0.5, -0.5, -0.5, -0.5, -0.5, -0.5, -0.5, -0....   
drone_ob  [[0.012732854600000001, -267.921035, 48.026962...   

                                                         3   \
goal          [-59.922351462103364, -434.86144609719025, 0]   
actions   [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, ...   
rewards   [-0.5, -0.5, -0.5, -0.5, -0.5, -0.5, -0.5, -0....   
drone_ob  [[0.012732854600000001, -267.921035, 48.026962...   

                                                         4   \
goal  

In [3]:
df = df_explore
drone_observations_df = df.loc['drone_ob']
episode=3
episodes = df.loc['actions'].shape[0]
explore_df = pd.DataFrame()
small_df = pd.DataFrame()

# first_attack = list()
far_x=list()
end_y=list()
end_z=list()
episode_list=list()
latest_attack_episode = 0
latest_attack_index = 0
for ep_ix in range(episodes-1):
    episode = ep_ix+1
    sample_df = pd.DataFrame()
    actions = df.loc['actions'].iloc[episode]
    # search for fist attack in episdode 
#     attack_start = actions.index(1)
#     if attack_start > latest_attack_index:
#         latest_attack_index = attack_start
#         latest_attack_episode = episode
    sample_df['action'] = actions
    sample_df['reward'] = df.loc['rewards'].iloc[episode]
    sample_df['reward_acum'] = sample_df['reward'].cumsum()
    # first_attack.append(list(sample_df['action']).index(1)) # stores when the first attack happens in the episode
    episode_length = sample_df.shape[0]
    sample_df['step'] = list(range(episode_length))
    x=list()
    y=list()
    z=list()
    roll = list()
    pitch = list()
    yaw = list()
    # linear velocity
    vx = list()
    vy = list()
    vz = list()
    # angular velocity
    wx = list()
    wy = list()
    wz = list()
    # motors
    m1 = list()
    m2 = list()
    m3 = list()
    m4 = list()
    
    

    for i in range(len(drone_observations_df.iloc[episode])):
    # print(drone_observations_df.iloc[1][0])
        drone_state=drone_observations_df.iloc[episode][i]
        position=drone_state[0:3]
        quaternion=drone_state[3:7]
        rpy=drone_state[7:10]
        linear_v=drone_state[10:13]
        angular_v=drone_state[13:16]
        motors=drone_state[16:20]
        # new features: roll, pitch and yaw derivatives
#         diff_rpy=drone_state[20,23]
        
        x.append(position[0])
        y.append(position[1])
        z.append(position[2])
        roll.append(rpy[0])
        pitch.append(rpy[1])
        yaw.append(rpy[2])
        m1.append(motors[0])
        m2.append(motors[1])
        m3.append(motors[2])
        m4.append(motors[3])
        # additional observations
        vx.append(linear_v[0])
        vy.append(linear_v[1])
        vz.append(linear_v[2])
        wx.append(angular_v[0])
        wy.append(angular_v[1])
        wz.append(angular_v[2])
        
        
    sample_df['x'] = x
    sample_df['y'] = y
    sample_df['z'] = z
    sample_df['roll'] = roll
    sample_df['pitch'] = pitch
    sample_df['yaw'] = yaw
    sample_df['episode'] = [episode]*episode_length
    sample_df['m1'] = m1
    sample_df['m2'] = m2
    sample_df['m3'] = m3
    sample_df['m4'] = m4
    # adding velocity to the dataframe
    sample_df['vx'] = vx
    sample_df['vy'] = vy
    sample_df['vz'] = vz
    sample_df['wx'] = wx
    sample_df['wy'] = wy
    sample_df['wz'] = wz
    # new features:
    # 
    
    # filling up small_df 
    min_x = min(x)
    max_x = max(x)
    farX = max_x if abs(max_x) > abs(min_x) else min_x
    far_x.append(farX)
    end_y.append(y[-1])
    end_z.append(z[-1])
    episode_list.append(episode)
    

    explore_df = pd.concat([explore_df, sample_df])

small_df['x'] = far_x
small_df['y'] = end_y
small_df['z'] = end_z
small_df['episode'] = episode_list

print('latest attack on episode {}'.format(latest_attack_episode))

latest attack on episode 0


In [4]:
interesting_episodes = [i+1 for i in range(39)]
scope_df = explore_df.loc[explore_df['episode'].isin(interesting_episodes)]
# scope_df = small_df.loc[abs(small_df['x'])>0.3]
# scope_df = small_df
# print(scope_df.shape)
symbol_order = {0:['circle','x',18],1:['x','circle',18]}
# fig = px.line(scope_df,x='x', y='y', line_group='episode', color='episode', symbol='action', symbol_sequence=symbol_order[0], color_discrete_sequence=px.colors.qualitative.Prism)
fig = px.line(scope_df,x='x', y='y', color='episode', color_discrete_sequence=px.colors.qualitative.Prism)
fig.update_traces(mode='markers', marker_line_width=2, marker_size=10)

title_size=18
text_size=16
fig['layout'].update(plot_bgcolor='#FFFFFF', 
                      width=1000,
                      height=800, title='Drone furthest X position',titlefont=dict(size=title_size))
fig['layout']['yaxis'].update(tickfont=dict(size=text_size), 
                               titlefont=dict(size=title_size),
                               gridcolor='#E2E2E2',
                               showline=True, 
                               linewidth=1, 
                               linecolor='black',
                               mirror=True)
fig['layout']['xaxis'].update(tickfont=dict(size=text_size),
                                gridcolor='#E2E2E2',
                               showline=True, 
                               linewidth=1, 
                               linecolor='black',
                               mirror=True)

iplot(fig)

In [5]:
filtered_df = scope_df.loc[abs(scope_df['x'])>0.4]
print(filtered_df.shape)

(528, 21)


In [8]:
file_name = 'result-baseline.json'
baseline = pd.read_json(file_name)
# print(baseline)
original_baseline = baseline.loc[:,2]
baseline_df = episode_to_df(original_baseline)
# print(baseline_df.iloc[3:])
baseline_df['step'] = baseline_df['step']-2
baseline_df = baseline_df.iloc[2:]

file_name = 'result.json'
attacks = pd.read_json(file_name)
print(attacks)
attacks_df = pd.DataFrame()
# for ix in [3,4,5,6,7,8]:
#     at1 = episode_to_df(attacks.loc[:,ix],episode=ix*100000)
#     at1['step'] = at1['step']-6
#     attacks_df = pd.concat([attacks_df, at1.iloc[6:]])
    
# at1= episode_to_df(attacks.loc[:,4],episode=999999)
# at1['step'] = at1['step']-6
# attacks_df = pd.concat([attacks_df, at1.iloc[6:]])
# at1 = episode_to_df(attacks.loc[:,3],episode=888888)
# at1['step'] = at1['step']-6
# attacks_df = pd.concat([attacks_df, at1.iloc[6:]])


                                                    1  \
goal      [-35.98153743937287, -430.6748418287196, 0]   
actions                                            []   
rewards                                            []   
drone_ob                                           []   

                                                          2  \
goal           [-103.16894138732384, -386.1456878474638, 0]   
actions                                                 [0]   
rewards                                              [-0.5]   
drone_ob  [[0.012732854600000001, -267.921035, 48.026962...   

                                                          3  \
goal           [23.456171668797282, -493.23841041770845, 0]   
actions                                                 [0]   
rewards                                              [-0.5]   
drone_ob  [[0.012732854600000001, -267.921035, 48.026962...   

                                                          4  \
goal            [-7

In [9]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# interesting_episodes = [0] + [300000,400000,500000,600000,700000,800000] # + [latest_attack_episode] # [4, 600, 8534, 8801, 12037, 12919, 33877] 
interesting_episodes = [i+1 for i in range(37)]
interesting_episodes = [7,12]
plot_df = explore_df.loc[explore_df['episode'].isin(interesting_episodes)]
plot_df = pd.concat([baseline_df,plot_df])
plot_df = pd.concat([plot_df, attacks_df])
plot_df['y'] = plot_df['y']*-1
plot_df['y'] = plot_df['y']-267.921
symbol_order = {0:['circle','x',18],1:['x','circle',18]}
fig2 = px.line(plot_df,x='x', y='y', line_group='episode', color='episode', symbol='action', 
               symbol_sequence=symbol_order[0], color_discrete_sequence=px.colors.qualitative.Dark2)
fig2.update_traces(mode='markers', marker_line_width=2, marker_size=20,selector={"marker_symbol" : 'x'})
fig2.update_traces(mode='markers', marker_line_width=2, marker_size=5,selector={"marker_symbol" : 'circle'})
layout = go.Layout( autosize=True, margin={'l': 0, 'r': 0, 't': 0, 'b': 0})
fig2['layout'].update(layout)
fig2['layout'].update(plot_bgcolor='#FFFFFF', 
                      width=800,
                      height=600, #title='Drone trajectory',t
                      titlefont=dict(size=title_size),showlegend=False)
fig2['layout']['yaxis'].update(tickfont=dict(size=text_size), 
                                   title='y [meters]',
                                   titlefont=dict(size=title_size),
                                   gridcolor='#E2E2E2',
                                   showline=True, 
                                   linewidth=1, 
                                   linecolor='black',
                                   mirror=True)
fig2['layout']['xaxis'].update(tickfont=dict(size=text_size),
                               title='x [meters]',
                               gridcolor='#E2E2E2',
                               showline=True, 
                               linewidth=1, 
                               linecolor='black',
                               mirror=True)

fig2['layout'].update(plot_bgcolor='#FFFFFF', width=800, height=450,
                         font=dict(family="Serif", 
                              color='black',
                              size=text_size)
                              )


# fig2 = fig2.add_vline(x=35.27048400479107)
# fig2 = fig2.add_vline(x=-39.76946454091133)
iplot(fig2)

# pl1 = ['reward', 'reward_acum']
# pl2 = ['x','y','z']
# pl3 = ['roll','pitch','yaw']
# pl4 = ['m1','m2','m3','m4']
# pl5 = ['vx', 'vy', 'vz']
# pl6 = ['wx', 'wy', 'wz']
# plotting_set = pl1 + pl2 + pl3 + pl5+ pl6 + pl4
# diff_set = []
# specs_data = list()
# # for pl_d in plotting_set:
# #     if pl_d in diff_set:
# #         specs_data.append([{"secondary_y": True}])
# #     else:
# #         specs_data.append([{"secondary_y": False}])


# fign = make_subplots(rows=len(plotting_set)+len(diff_set), cols=1,
#                     shared_xaxes=True,
#                     vertical_spacing=0.02,
# #                     specs=specs_data,
#                     )

# colors = px.colors.qualitative.Bold
# colors = colors + colors + colors + colors
# print(len(colors))
# show_legend = [True]*len(interesting_episodes)
# show_legend_diff = [True]*len(interesting_episodes)
# baseline_plot = plot_df.loc[plot_df['episode']==0]
# ix = 0
# plot_delta = 0
# plot_labels = list()
# for _, pls in enumerate(plotting_set):
#     ix = ix+1+plot_delta
#     plot_labels.append(pls)
#     for iex, iep in enumerate(interesting_episodes):
#         full_this_plot = plot_df.loc[plot_df['episode']==iep]
#         for act in [0,1]:
#             this_plot = full_this_plot.loc[full_this_plot['action']==act]
#             fign.add_trace(go.Scatter(x=this_plot['step'], y=this_plot[pls], 
#                     name=str(iep)+'-'+str(act), legendgroup=str(iep)+'-'+str(act), showlegend=show_legend[iex], 
#                     marker={'color':colors[iex],'symbol':symbol_order[0][act]}, mode='markers'), row=ix, col=1)
#         plot_delta = 0
#         if pls in diff_set:
#             diff_label = pls+'-diff'
#             if not diff_label in plot_labels:
#                 plot_labels.append(diff_label)
#             plot_delta = 1
#             y_delta = full_this_plot[pls]-baseline_plot[pls]
#             y_delta = y_delta.fillna(0)
#             fign.add_trace(go.Scatter(y=y_delta, 
#                     name=str(iep)+'-diff', legendgroup=str(iep)+'-diff', showlegend=show_legend_diff[iex],
#                     mode='lines', line={'color':colors[iex]}), 
#                 row=ix+1, col=1)
#             show_legend_diff[iex]=False

#         show_legend[iex]=False


# # Formatting 
# fign['layout'].update(plot_bgcolor='#FFFFFF', 
#                       width=800,
#                       height=300*len(plotting_set), title='All plots',titlefont=dict(size=title_size))
# fign.update_traces(marker_line_width=2, marker_size=10)

# for ix,plt_name in enumerate(plot_labels):
#     fign['layout']['yaxis'+str(ix+1)].update(title=plt_name,
#                                    tickfont=dict(size=text_size), 
#                                    titlefont=dict(size=title_size),
#                                    gridcolor='#E2E2E2',
#                                    showline=True, 
#                                    linewidth=1, 
#                                    linecolor='black',
#                                    mirror=True)
#     fign['layout']['xaxis'+str(ix+1)].update(tickfont=dict(size=text_size),
#                                gridcolor='#E2E2E2',
#                                showline=True, 
#                                linewidth=1, 
#                                linecolor='black',
#                                mirror=True)
# fign['layout']['xaxis'+str(len(plotting_set))].update(title='Step')
# # fign.update_xaxes(matches='x')



# iplot(fign)

In [ ]:
fig2.write_image('turn.pdf')

from sklearn.cluster import KMeans

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

#Getting unique labels
 
u_labels = np.unique(labels)
 
#plotting the results:
 
for i in u_labels:
    plt.scatter(small_df[cols].values[labels == i , 0] , small_df[cols].values[labels == i , 1] , label = i)
plt.legend()
plt.show()


In [ ]:
sample_df